# Recommenders 2 -- (45min) 

## Goals of this practical:

1. Understand/Implement two common ranking metrics (~15min)
2. See how SVD model perform w/ respect to those metrics (~15min)
3. Understand the difference between implicit/explicit CF (~5min)
3. Know and implement a really simple ranking baseline (~5min)
4. Use a library to build an implicit recommender sysem (~5min)



## Data used : [smallest movie-lens dataset](https://grouplens.org/datasets/movielens/)

In this practical we use a small dataset of user ratings on movies. Specifically, we treat the dataset as list of $(user,item,rating)$ triplets.






In [5]:
## Uncomment this to install required packages if needed (and restart kernel !)
#! pip install --upgrade pandas
#! pip install --upgrade seaborn
#! pip install --upgrade scikit-surprise
#! pip install --upgrade numpy
#! pip install --upgrade lightfm

In [6]:
import numpy as np # For array things
import pandas as pd # For handling data

## Loading Data  & creating train/test (same as before)

=> We still consider 20% as test

In [7]:
ratings = pd.read_csv("./BDD/avis.csv")
#ratings = ratings.drop(['url','date_published','treated', 'review_href'], axis=1)
ratings.head(5)

,Unnamed: 0,author,title_review,note,title,comment
0,0,Monsieur Guillaume,Voyages sur les ailes des papillons,8,Mariposas,"Lorsque le jeu est jeu, bon, réflexif, joli po..."
1,1,morlockbob,le festival de Michoacan,7,Mariposas,Comment continuer après un mega hit ? Simpleme...
2,2,SwatSh,Vivez la migration des monarques,7,Mariposas,"Vin d'jeu: Avec Mariposas, Elizabeth Hargrave ..."
3,3,Timi JeuxATheme,Bon,8,Mariposas,NaN
4,4,prunelles,Envolez-moi,9,Mariposas,"Très joli bijou que ce jeu-là ! Le matériel, l..."


In [8]:
train_indexes,test_indexes = [],[]

for index in range(len(ratings)):
    if index%5 == 0:
        test_indexes.append(index)
    else:
        train_indexes.append(index)

train_ratings = ratings.iloc[train_indexes].copy()
test_ratings = ratings.iloc[test_indexes].copy()


# Collaborative Filtering and Ranking Metrics

Most of the time recommender systems present $k$ items to the users. Therefore, recommenders are often evaluated by how many relevant items are in their $k$ set (using ranking metrics).

Different ranking methods exists such as:
- [Mean Reciprocal Rank](http://en.wikipedia.org/wiki/Mean_reciprocal_rank) 
- [normalized Discounted Cumulative Gain](https://en.wikipedia.org/wiki/Discounted_cumulative_gain)


## Mean Reciprocal Rank :

From [Wikipedia](https://en.wikipedia.org/wiki/Mean_reciprocal_rank):

> The mean reciprocal rank is a statistic measure for evaluating any process that produces a list of possible responses to a sample of queries, ordered by probability of correctness. The reciprocal rank of a query response is the multiplicative inverse of the rank of the first correct answer: 1 for first place, $\frac{1}{2}$ for second place, $\frac{1}{3}$ for third place and so on. The mean reciprocal rank is the average of the reciprocal ranks of results for a sample of queries Q

$$ MRR = \frac{1}{|Q|}\sum^{|Q|}_{i=1}\frac{1}{\text{rank}_i} $$





This metric can be used to assess the mean rank of relevant suggestions

## TODO: Implement MRR

To compute MRR, we need relevance lists which encodes whether or not items are relevant: 

- it's a list of 0's or 1's respectively meaning 'not relevant' and 'relevant'
- In the MRR case, we're just looking for the 1st relevant item


In [9]:
test_list = [[0,0,1],[0,1,0],[1,0,0],[0,0,0]]

def rr(list_items):
    relevant_indexes = np.asarray(list_items).nonzero()[0]
    
    if len(relevant_indexes) > 0:
        return 1/(relevant_indexes[0]+1) # arrays are indexed from 0
    else:
        return 0

def mrr(list_list_items):
    return np.mean([rr(list_item) for list_item in list_list_items])

mrr(test_list) #0.4583333333333333

0.4583333333333333

## (normalized) Discounted Cumulative Gain:

From [Wikipedia](https://en.wikipedia.org/wiki/Discounted_cumulative_gain):

> Discounted cumulative gain (DCG) is a measure of ranking quality. In information retrieval, it is often used to measure effectiveness of web search engine algorithms or related applications. Using a graded relevance scale of documents in a search-engine result set, DCG measures the usefulness, or gain, of a document based on its position in the result list. The gain is accumulated from the top of the result list to the bottom, with the gain of each result discounted at lower ranks.

$$DCG_p = \sum^p_{i=1}\frac{rel_i}{\log_2{(i+1)}} = rel_1 + \sum^p_{i=2}\frac{rel_i}{\log_2{(i+1)}}$$

>Two assumptions are made in using DCG and its related measures.
- Highly relevant documents are more useful when appearing earlier in a search engine result list (have higher ranks)
- Highly relevant documents are more useful than marginally relevant documents, which are in turn more useful than non-relevant documents.

## TODO: Implement DCG

In [10]:
# The dcg@k is the sum of the relevance, penalized gradually
def dcg_at_k(r, k):
    """Score is discounted cumulative gain (dcg)
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        
    """
    r = np.asfarray(r)[:k]
    if r.size:
        return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        
    return 0.

# test values
# r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
# dcg_at_k(r, 1) => 3.0
# dcg_at_k(r, 2) => 4.2618595071429155
r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
print(dcg_at_k(r, 1))
print(dcg_at_k(r, 2))

3.0
4.2618595071429155


> Result lists vary in length. Comparing performance from one query to the next cannot be consistently achieved using DCG alone, so the cumulative gain at each position for a chosen value of p should be normalized. The normalized discounted cumulative gain, or nDCG, is computed as: 

$$ nDCG_p = \frac{DCG_p}{IDCG_p} $$

> Where IDCG_p is the maximum possible DCG through position p, also called Ideal DCG (IDCG). It is obtained by sorting all relevant documents in the corpus by their relative relevance and computing the DCG:

$$ IDCG_p = max(DCG_p) $$


## (TODO) Implement NDCG

In [11]:

# And it's normalized version
def ndcg_at_k(r, k):
    """
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
    """
    dcg_max =  dcg_at_k(sorted(r)[::-1],k) 
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k) / dcg_max

# test values
# r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
# ndcg_at_k(r, 1) => 1.0
# ndcg_at_k(r, 4) => 0.794285
    
r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]    
print(ndcg_at_k(r,1))
ndcg_at_k(r, 4)   

1.0


0.7942854176010882

# Now, let's see how SVD performs on these metrics:

Here, we're not in the same evaluation framework as in the last session. The goal for our recommender is to present $k$ items to the users instead of simply scoring $(user,item)$ pairs. We can still use previous models if we consider the predicted rating as a way to rank item to present.

###  To make a recommender we also need two `{userId: [movieId, movieId, ...] }` dictionnaries:

- `already_seen`: Items that were already seen by users. This is for training and not recommending them again
- `ground_truth`: Items that will be seen and liked (rating >= 5) by users. This is our ground truth to evaluate our predictions.

In [12]:
already_seen = (
    train_ratings 
    .groupby("author")["title"]
    .apply(list)
    .to_dict()
    )

ground_truth = (
    test_ratings[test_ratings.note >= 8] 
    .groupby("author")["title"]
    .apply(list)
    .to_dict()
    )

### We also need the set of all items that can be recommended

In [13]:
existing_items = set(train_ratings["title"].unique())
print("The recommender system will have to pick a few items from",len(existing_items),"possible items")

The recommender system will have to pick a few items from 3306 possible items


## (TODO) Ok, now, let's make a quick surprise SVD recommender

In [14]:

from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate

data = Dataset.load_from_df(train_ratings[['author', 'title', 'note']], Reader(rating_scale=(0, 10)))
model = SVD()

# training a quick model
model.fit(data.build_full_trainset())


#### Function to make mse predictions:

In [15]:
def svd_rating_pred(user_item):
    user = user_item["author"]
    item = user_item["title"]
    
    prediction = model.predict(user,item)
    
    return prediction.est

test_ratings["svd_prediction"] = test_ratings[["author","title"]].apply(svd_rating_pred,axis=1) 

mse = ((test_ratings["note"] - test_ratings["svd_prediction"])**2).mean()
mae = ((test_ratings["note"] - test_ratings["svd_prediction"]).abs()).mean()

print(f"MSE: {mse} -- MAE: {mae}")

MSE: 3.4309851890243444 -- MAE: 1.424524244021255


#### (todo) function to make predictions given `(surprise model,user,item)`

In [16]:
def model_rating_pred(model,user,item):
    prediction = model.predict(user,item)
    return prediction.est

## Let's create the relevance list for our MRR function

In [17]:
list_of_rel = []

for user,will_see in ground_truth.items():
    rel_list = []
    will_see = set(will_see)
    has_seen = set(already_seen.get(user, []))
    can_see = [(mid,model_rating_pred(model,user,mid)) for mid in existing_items - has_seen]
    
    
    for movie,score in reversed(sorted(can_see,key=lambda x:x[1])):
        if movie in will_see:
            rel_list.append(1)
            break
        else:
            rel_list.append(0)        
    rel_list[-1] = 1 # when no relevant item exist
    list_of_rel.append(rel_list)
    

svd_mrr = mrr(list_of_rel)


In [18]:
f"On average, the {int(round(1/svd_mrr,0))}th proposed item is relevant (on {len(existing_items)})"

'On average, the 27th proposed item is relevant (on 3306)'

Our result is the following :
> 'On average, the 9th proposed item is relevant (on 8970)'

So, on average, if you show around 10 items to someone, there will be at least one which will be relevant: not bad for a guess on 8970 !

### We need something better to compare though:

## A good enough implicit baseline: popular items
What if you only show popular items to people ?

### (TODO) let's find the popular items:

- By popular we mean the "most" rated movies (the one with the more rating)

should be something along this: 
[318,
 356,
 296,
 2571,
 593,
 110,
 480,
 260,
 589,
 527,
 780,
 ...
 ]

In [19]:
movie_counts = train_ratings.groupby("title")["note"].count().sort_values(ascending=False)
popular_item_list = list(movie_counts.index)

In [20]:
print(len(popular_item_list))

3306


In [21]:
#all_items = set(popular_item_list)

## (TODO) Building the popular recommendation relevance list per user.

To use our MRR metric function, we need to build a list of lists containing 0's or 1's. There is one list per user.
0 means not relevant, 1 means relevant. We need to score each item until the first relevant one.

In [22]:
list_of_rel = []

for user,will_see in ground_truth.items():
    rel_list = []
    will_see = set(will_see)
    has_seen = set(already_seen.get(user, []))
    
    for movie in popular_item_list:
        if movie in has_seen:         # User has already seen movie -> Can filter prediction
            continue
        elif movie in will_see:       # User will see, spot on suggestion !         
            rel_list.append(1)
            break
        else:                         # No clue.
            rel_list.append(0)
            
    if rel_list[-1] == 1:             # when no relevant item exist, no need to take it into account.
        list_of_rel.append(rel_list)
    

In [23]:
pop_mrr = mrr(list_of_rel)
f"On average, the {int(round(1/pop_mrr,0))}th proposed item is relevant (on {len(existing_items)})"

'On average, the 26th proposed item is relevant (on 3306)'

Our result is the following :
> 'On average, the 4th proposed item is relevant (on 8970)'

Ok, the popular baseline is WAY better than our SVD model...

## Implicit Collaborative Filtering 

Implicit collaborative filtering, unlike explicit CF, is the task of learning from interactions (not ratings). The models are quite close but mainly differ by the optimized loss.



Here, we propose to use the [lightFM](https://github.com/lyst/lightfm) model to do implicit collaborative filtering.


> LightFM is a Python implementation of a number of popular recommendation algorithms for both implicit and explicit feedback, including efficient implementation of BPR and WARP ranking losses. It's easy to use, fast (via multithreaded model estimation), and produces high quality results.

LightFM is a simple matrix factorization algorithm where a rating is predicted in the following way:

$$\hat{r_{ui}} = f(q_u · p_i + b_u + b_i)$$

Where $q_u$,$p_i$ and $b_u$, $b_i$ are respectively user and item latent profile and features

### What are interactions :

Interactions can be any type (clicks, pause, gaze, comment, review) it's a lot more versatile than ratings. Here we'll consider two setups:

- Interactions are ratings >= 5
- Interactions are every ratings 


## (a) Let's create the interaction train/test dataset within the framework

[Relevant Documentation](http://lyst.github.io/lightfm/docs/lightfm.data.html)

To create a dataset: 
- (a) Create an instance of the Dataset class.
- (b) Call fit (or fit_partial), supplying user/item ids and feature names that you want to use in your model. This will create internal mappings that translate the ids and feature names to internal indices used by the LightFM model.
- (c) Call build_interactions with an iterable of (user id, item id) or (user id, item id, weight) to build an interactions and weights matrix.

In [24]:
from lightfm.data import Dataset

# (a) Create a dataset
dataset = Dataset()


# (b) Create an internal mapping for users and items (We need to consider train + test)
dataset.fit((x for x in ratings["author"]),
            (x for x in ratings['title']))

# (c) Create the interaction matrices
(train_interactions, weights) = dataset.build_interactions(
    ((x.author, x.title) for x in train_ratings.itertuples() if x.note >= 8) # We only consider 5's as interactions
) 
(test_interactions, weights) = dataset.build_interactions(
    ((x.author, x.title) for x in test_ratings.itertuples() if x.note >= 8)  # We only consider 5's as interactions
) 

C:\Users\hacho\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


## (b) Create and train the lightFM model


The model learns embeddings (latent representations in a high-dimensional space) for users and items in a way that encodes user preferences over items. When multiplied together, these representations produce scores for every item for a given user; items scored highly are more likely to be interesting to the user.

Four loss functions are available:

- logistic: useful when both positive (1) and negative (-1) interactions are present.
- BPR: Bayesian Personalised Ranking [1] pairwise loss. Maximises the prediction difference between a positive example and a randomly chosen negative example. Useful when only positive interactions are present and optimising ROC AUC is desired.
- WARP: Weighted Approximate-Rank Pairwise [2] loss. Maximises the rank of positive examples by repeatedly sampling negative examples until rank violating one is found. Useful when only positive interactions are present and optimising the top of the recommendation list (precision@k) is desired.
- k-OS WARP: k-th order statistic loss [3]. A modification of WARP that uses the k-th positive example for any given user as a basis for pairwise updates.


In [25]:
from lightfm import LightFM

model = LightFM(loss='bpr',random_state=50000)
model.fit(train_interactions)

: 

## (c) Evaluation of learnt model with lightfm

[Relevant documentation](http://lyst.github.io/lightfm/docs/lightfm.evaluation.html)

In [ ]:
from lightfm.evaluation import reciprocal_rank
bpr_mrr = reciprocal_rank(model, test_interactions, train_interactions).mean()

ValueError: Test interactions matrix and train interactions matrix share 23 interactions. This will cause incorrect evaluation, check your data split.

In [ ]:
f"On average, the {int(round(1/bpr_mrr,0))}th proposed item is relevant (on {len(existing_items)})"

Now, we get
> 'On average, the 6th proposed item is relevant (on 8970)'

It's much better than the SVD, but still worse than the popular baseline

## (TODO) Let's do the same, but now we consider EVERY rating as one interaction.

In [ ]:
# Create the interaction matrix
(train_interactions_all, weights) = dataset.build_interactions(((x.author, x.title) for x in train_ratings.itertuples())) # We consider every interactions
(test_interactions_all, weights) = dataset.build_interactions(((x.author, x.title) for x in test_ratings.itertuples())) # We consider every interactions

from lightfm import LightFM

#
model_bpr_all = LightFM(loss='bpr',random_state=50000)
model_bpr_all.fit(train_interactions_all)

bpr_mrr_all = reciprocal_rank(model_bpr_all, test_interactions_all, train_interactions_all).mean()

ValueError: Test interactions matrix and train interactions matrix share 40 interactions. This will cause incorrect evaluation, check your data split.

In [ ]:
f"On average, the {int(round(1/bpr_mrr_all,0))}th proposed item is relevant (on {len(existing_items)})"

Now we got
> 'On average, the 3th proposed item is relevant (on 8970)'

Much better !

### Some time left ? Try and experiment with every lightfm losses 